In [23]:
# Packages for analysis
from sqlalchemy import create_engine
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.utils import resample

# Packages for visuals
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(font_scale=1.2)

# Allows charts to appear in the notebook
%matplotlib inline

# Pickle package
import pickle

## Uploading the data

In [24]:
cnx = create_engine('postgresql://kaitlinpuryear@54.196.215.198:5432/kaitlinpuryear')
df = pd.read_sql_query('''SELECT * FROM noshows3''',cnx)

In [25]:
df=df.drop(labels=["index"], axis=1)

In [26]:
df['2+problems'] = (df['hypertension']+df['diabetes']+df['alcoholism']+df['handicap']>2)

/anaconda3/lib/python3.6/site-packages/pandas/core/computation/expressions.py:179: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [27]:
# Separate majority (True) and minority (False) classes

majority = df[df.showed_up==True]
minority = df[df.showed_up==False]
 
# upsample minority class
minority_upsampled = resample(minority, 
                                 replace=True,        # sample without replacement
                                 n_samples=88208,     # to match majority class
                                 random_state=123)    # reproducible results
 
# Combine majority class with upsampled minority class
upsampled = pd.concat([minority_upsampled, majority])
 
# Display new class counts
upsampled.showed_up.value_counts()

True     88208
False    88208
Name: showed_up, dtype: int64

In [28]:
X0 = df.drop(['showed_up'], axis=1)
y0 = df["showed_up"]

In [29]:
X0.head()

,age,scholarship,hypertension,diabetes,alcoholism,handicap,text_sent,woman,delta,2+problems
0,62,False,True,False,False,False,False,True,0,False
1,56,False,False,False,False,False,False,False,0,False
2,62,False,False,False,False,False,False,True,0,False
3,8,False,False,False,False,False,False,True,0,False
4,56,False,True,True,False,False,False,True,0,False


## Fit Model

In [30]:
model = RandomForestClassifier(class_weight = 'balanced', max_depth=20, n_estimators=100, random_state=76)
model.fit(X0, y0)


RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=20, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=76,
            verbose=0, warm_start=False)

In [84]:
def patients(age, scholarship, text_sent, woman, delta, problems):
    return (model.predict_proba([[age,scholarship,False,False,False,False,text_sent,woman,delta,problems]]))

In [85]:
#patients(26, False,False,False, 100, True)

array([[ 0.4589517,  0.5410483]])

In [32]:
noshow_dict = {'noshow_model': randomforest,'noshow_features': ['age','scholarship','hypertension',
                                           'diabetes','alcoholism','handicap',
                                           'text_sent','showed_up','woman',
                                           'delta','2+problems'] }

In [33]:
noshow_dict

{'noshow_features': ['age',
  'scholarship',
  'hypertension',
  'diabetes',
  'alcoholism',
  'handicap',
  'text_sent',
  'showed_up',
  'woman',
  'delta',
  '2+problems'],
 'noshow_model': RandomForestClassifier(bootstrap=True, class_weight='balanced',
             criterion='gini', max_depth=20, max_features='auto',
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_jobs=1, oob_score=False, random_state=76,
             verbose=0, warm_start=False)}

## Pickle the model

In [34]:
# Pickle
pickle.dump(model, open("noshow.p", "wb"))